In [2]:
import pandas as pd
import numpy as np
from ScanImageTiffReader import ScanImageTiffReader
import os
import matplotlib.pyplot as plt
import scipy.signal
import tifffile
from scipy.signal import butter, lfilter,find_peaks, gaussian, convolve
from ibllib.io.video import *
import scipy.ndimage
import cv2
from matplotlib.animation import FuncAnimation, FFMpegWriter
from IPython.display import HTML
from matplotlib.animation import PillowWriter
from ScanImageTiffReader import ScanImageTiffReader
import json
import ast
from murphlib.caimage import *
from murphlib.tools import *
from murphlib.plotting import *
from murphlib.regression import *
from sklearn.decomposition import PCA
from sklearn.metrics import explained_variance_score
from scipy.stats import zscore

plt.rcParams['animation.ffmpeg_path'] = r'D:\ffmpeg\bin\ffmpeg.exe'

In [ ]:
paths = [
    #  r'Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_R2\20240815',
    #  r'Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_R2\20240821',
     #r'Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_R3\20240904',
     r'Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_R3\20240910',
     r'Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_R3\20240918',
     r'Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_R3\20240924',
    #  r'Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_R2\20241015',
     r'Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_R3\20241018',
    #  r'Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_R2\20241106',
     r'Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_R3\20241111',
    #  #r'Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_L2\20241115',
    #  r'Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_R2\20241118',
    r'Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_R3\20241119',
     
]




for path in paths:
    subdir = [d for d in os.listdir(path)]
    print('processing '+path)
    if 'arm1' in subdir:
        arm1_img_path = os.path.join(path, 'arm1')
        try: 
            with open(os.path.join(arm1_img_path, 'ca_img_wholefile_metadata.json'), 'r') as json_file:
                arm1_img_meta = json.load(json_file)
        except:
            raw_img_path = find_tif_file(arm1_img_path)
            reader=ScanImageTiffReader(raw_img_path)
            arm1_img_meta = parse_text_to_dict(reader.metadata())
            with open(os.path.join(arm1_img_path, 'ca_img_wholefile_metadata.json'), 'w') as json_file:
                json.dump(arm1_img_meta, json_file)
        arm1_suite2p_path = os.path.join(arm1_img_path,'suite2p','plane0')
        arm1_fluo = np.load(arm1_suite2p_path+'\\F.npy', allow_pickle=True)
        arm1_neufluo = np.load(arm1_suite2p_path+'\\Fneu.npy', allow_pickle=True)
        raw_arm1_stat = np.load(arm1_suite2p_path+'\\stat.npy', allow_pickle=True)
        arm1_iscell = np.load(arm1_suite2p_path+'\\iscell.npy', allow_pickle=True)
        arm1_iscell = arm1_iscell[:,0]==1
        arm1_stat = []
        for i, s in enumerate(raw_arm1_stat):
            if arm1_iscell[i]:
                s['id'] = i
                arm1_stat.append(s)
        arm1_fluo = arm1_fluo[arm1_iscell,:]
        arm1_neufluo = arm1_neufluo[arm1_iscell,:]
        frame_rate_arm1 = arm1_img_meta['SI.hRoiManager.scanFrameRate']
        try:
            arm1_dff = np.load(os.path.join(arm1_img_path, 'dff_moving_percentile.npy'))
        except:
            arm1_dff = calculate_dff_with_moving_percentile(arm1_fluo, frame_rate=frame_rate_arm1, moving_window=20, percentile=10)
            #### only do this if front need to be padded
            #arm1_dff = np.hstack([np.zeros((arm1_dff.shape[0],30)), arm1_dff])
            np.save(os.path.join(arm1_img_path, 'dff_moving_percentile.npy'), arm1_dff)

        arm1_aspect_ratio = np.array([arm1_stat[i]['aspect_ratio'] for i in range(len(arm1_stat))])
        arm1_npix = np.array([arm1_stat[i]['npix'] for i in range(len(arm1_stat))])
        arm1_pc_idx = np.where((arm1_aspect_ratio<1.1) & (arm1_npix<120) & (arm1_npix>100))[0]
        arm1_dd_idx = np.where((arm1_aspect_ratio>1.1) | (arm1_npix>120) | (arm1_npix<100))[0]
        arm1_pc_suite2p_idx = np.array([arm1_stat[i]['id'] for i in arm1_pc_idx])
        arm1_dd_suite2p_idx = np.array([arm1_stat[i]['id'] for i in arm1_dd_idx])

        try:
            arm1_dff = np.load(os.path.join(arm1_img_path, 'dff_moving_percentile_pc_lowpass_0p5hz.npy'))
        except:
            lp_arm1_dff = arm1_dff.copy()
            if len(arm1_pc_idx) != 0:
                arm1_pc_dff = calculate_dff_with_moving_percentile(arm1_fluo[arm1_pc_idx], frame_rate=frame_rate_arm1, moving_window=60, percentile=50)
                lp_arm1_dff[arm1_pc_idx] = arm1_pc_dff
            lp_arm1_dff[arm1_pc_idx] = butter_filter(lp_arm1_dff[arm1_pc_idx], cutoff=0.5, fs=frame_rate_arm1, filter_type='low', order=5)
            np.save(os.path.join(arm1_img_path, 'dff_moving_percentile_pc_lowpass_0p5hz.npy'), lp_arm1_dff)


        frames_metadata_export_path = os.path.join(arm1_img_path, 'ca_img_frames_metadata.json')
        img_path = find_tif_file(arm1_img_path)
        try:
            with open(frames_metadata_export_path, 'r') as json_file:
                frames_metadata = json.load(json_file)
        except:
            
            reader=ScanImageTiffReader(img_path)
            frames_metadata = []

            with ScanImageTiffReader(img_path) as reader:
                num_frames = reader.shape()[0]  # Assuming the first dimension represents frames
                
                for i in range(num_frames):
                    description = reader.description(i)
                    frame_metadata = {}  # Dictionary to hold this frame's metadata
                    
                    for line in description.split('\n'):
                        if '=' in line:  # Ensure there's a key-value pair to parse
                            key, value = line.split('=', 1)  # Split by the first '='
                            key = key.strip()
                            
                            try:
                                # Safely evaluate the value string to the appropriate Python data type
                                # This works for numbers, lists, dictionaries, etc.
                                value = ast.literal_eval(value.strip())
                            except (ValueError, SyntaxError):
                                # If evaluation fails, keep the value as a string
                                value = value.strip()
                            
                            frame_metadata[key] = value
                    
                    frames_metadata.append(frame_metadata)
                
            with open(frames_metadata_export_path, 'w') as json_file:
                json.dump(frames_metadata, json_file)
    if 'arm2' in subdir:
        arm2_img_path = os.path.join(path, 'arm2')
        try: 
            with open(os.path.join(arm2_img_path, 'ca_img_wholefile_metadata.json'), 'r') as json_file:
                arm2_img_meta = json.load(json_file)
        except:
            raw_img_path = find_tif_file(arm2_img_path)
            reader=ScanImageTiffReader(raw_img_path)
            arm2_img_meta = parse_text_to_dict(reader.metadata())
            with open(os.path.join(arm2_img_path, 'ca_img_wholefile_metadata.json'), 'w') as json_file:
                json.dump(arm2_img_meta, json_file)
        arm2_suite2p_path = os.path.join(arm2_img_path,'suite2p','plane0')
        arm2_fluo = np.load(arm2_suite2p_path+'\\F.npy', allow_pickle=True)
        arm2_neufluo = np.load(arm2_suite2p_path+'\\Fneu.npy', allow_pickle=True)
        raw_arm2_stat = np.load(arm2_suite2p_path+'\\stat.npy', allow_pickle=True)
        arm2_iscell = np.load(arm2_suite2p_path+'\\iscell.npy', allow_pickle=True)
        arm2_iscell = arm2_iscell[:,0]==1
        arm2_stat = []
        for i, s in enumerate(raw_arm2_stat):
            if arm2_iscell[i]:
                s['id'] = i
                arm2_stat.append(s)
        arm2_fluo = arm2_fluo[arm2_iscell,:]
        arm2_neufluo = arm2_neufluo[arm2_iscell,:]
        frame_rate_arm2 = arm2_img_meta['SI.hRoiManager.scanFrameRate']
        try:
            arm2_dff = np.load(os.path.join(arm2_img_path, 'dff_moving_percentile.npy'))
        except:
            arm2_dff = calculate_dff_with_moving_percentile(arm2_fluo, frame_rate=frame_rate_arm2, moving_window=20, percentile=10)
            #### only do this if front need to be padded
            #arm2_dff = np.hstack([np.zeros((arm2_dff.shape[0],30)), arm2_dff])
            np.save(os.path.join(arm2_img_path, 'dff_moving_percentile.npy'), arm2_dff)
        

        arm2_aspect_ratio = np.array([arm2_stat[i]['aspect_ratio'] for i in range(len(arm2_stat))])
        arm2_npix = np.array([arm2_stat[i]['npix'] for i in range(len(arm2_stat))])
        arm2_pc_idx = np.where((arm2_aspect_ratio<1.1) & (arm2_npix<120) & (arm2_npix>100))[0]
        arm2_dd_idx = np.where((arm2_aspect_ratio>1.1) | (arm2_npix>120) | (arm2_npix<100))[0]
        arm2_pc_suite2p_idx = np.array([arm2_stat[i]['id'] for i in arm2_pc_idx])
        arm2_dd_suite2p_idx = np.array([arm2_stat[i]['id'] for i in arm2_dd_idx])

        try:
            arm2_dff = np.load(os.path.join(arm2_img_path, 'dff_moving_percentile_pc_lowpass_0p5hz.npy'))
        except:
            lp_arm2_dff = arm2_dff.copy()
            if len(arm2_pc_idx) != 0:
                arm2_pc_dff = calculate_dff_with_moving_percentile(arm2_fluo[arm2_pc_idx], frame_rate=frame_rate_arm2, moving_window=60, percentile=50)
                lp_arm2_dff[arm2_pc_idx] = arm2_pc_dff
            lp_arm2_dff[arm2_pc_idx] = butter_filter(lp_arm2_dff[arm2_pc_idx], cutoff=0.5, fs=frame_rate_arm2, filter_type='low', order=5)
            np.save(os.path.join(arm2_img_path, 'dff_moving_percentile_pc_lowpass_0p5hz.npy'), lp_arm2_dff)


        frames_metadata_export_path = os.path.join(arm2_img_path, 'ca_img_frames_metadata.json')
        img_path = find_tif_file(arm2_img_path)
        try:
            with open(frames_metadata_export_path, 'r') as json_file:
                frames_metadata = json.load(json_file)
        except:
            
            reader=ScanImageTiffReader(img_path)
            frames_metadata = []

            with ScanImageTiffReader(img_path) as reader:
                num_frames = reader.shape()[0]  # Assuming the first dimension represents frames
                
                for i in range(num_frames):
                    description = reader.description(i)
                    frame_metadata = {}  # Dictionary to hold this frame's metadata
                    
                    for line in description.split('\n'):
                        if '=' in line:  # Ensure there's a key-value pair to parse
                            key, value = line.split('=', 1)  # Split by the first '='
                            key = key.strip()
                            
                            try:
                                # Safely evaluate the value string to the appropriate Python data type
                                # This works for numbers, lists, dictionaries, etc.
                                value = ast.literal_eval(value.strip())
                            except (ValueError, SyntaxError):
                                # If evaluation fails, keep the value as a string
                                value = value.strip()
                            
                            frame_metadata[key] = value
                    
                    frames_metadata.append(frame_metadata)
                
            with open(frames_metadata_export_path, 'w') as json_file:
                json.dump(frames_metadata, json_file)

processing Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_R3\20240904
processing Z:\TM_Lab\Edward\Cerebellum_Imaging\Pcp2-jgcamp8m\A_R3\20240910


In [ ]:

arm2_aspect_ratio = np.array([arm2_stat[i]['aspect_ratio'] for i in range(len(arm2_stat))])
arm2_npix = np.array([arm2_stat[i]['npix'] for i in range(len(arm2_stat))])
arm2_pc_idx = np.where((arm2_aspect_ratio<1.1) & (arm2_npix<120) & (arm2_npix>100))[0]
arm2_dd_idx = np.where((arm2_aspect_ratio>1.1) | (arm2_npix>120) | (arm2_npix<100))[0]
arm2_pc_suite2p_idx = np.array([arm2_stat[i]['id'] for i in arm2_pc_idx])
arm2_dd_suite2p_idx = np.array([arm2_stat[i]['id'] for i in arm2_dd_idx])
try:
    arm1_dff = np.load(os.path.join(arm1_img_path, 'dff_moving_percentile_pc_lowpass_0p5hz.npy'))
    arm2_dff = np.load(os.path.join(arm2_img_path, 'dff_moving_percentile_pc_lowpass_0p5hz.npy'))
except:
    lp_arm1_dff = arm1_dff.copy()
    if len(arm1_pc_idx) != 0:
        arm1_pc_dff = calculate_dff_with_moving_percentile(arm1_fluo[arm1_pc_idx], frame_rate=frame_rate_arm1, moving_window=60, percentile=50)
        lp_arm1_dff[arm1_pc_idx] = arm1_pc_dff
    lp_arm1_dff[arm1_pc_idx] = butter_filter(lp_arm1_dff[arm1_pc_idx], cutoff=0.5, fs=frame_rate_arm1, filter_type='low', order=5)
    np.save(os.path.join(arm1_img_path, 'dff_moving_percentile_pc_lowpass_0p5hz.npy'), lp_arm1_dff)
    lp_arm2_dff = arm2_dff.copy()
    if len(arm2_pc_idx) != 0:
        arm2_pc_dff = calculate_dff_with_moving_percentile(arm2_fluo[arm2_pc_idx], frame_rate=frame_rate_arm2, moving_window=60, percentile=50)
        lp_arm2_dff[arm2_pc_idx] = arm2_pc_dff
    lp_arm2_dff[arm2_pc_idx] = butter_filter(lp_arm2_dff[arm2_pc_idx], cutoff=0.5, fs=frame_rate_arm2, filter_type='low', order=5)
    np.save(os.path.join(arm2_img_path, 'dff_moving_percentile_pc_lowpass_0p5hz.npy'), lp_arm2_dff)
    arm1_dff = np.load(os.path.join(arm1_img_path, 'dff_moving_percentile_pc_lowpass_0p5hz.npy'))
    arm2_dff = np.load(os.path.join(arm2_img_path, 'dff_moving_percentile_pc_lowpass_0p5hz.npy'))